In [1]:
from IPython.display import Markdown
import Twitter_API_Credentials as api_cred

In [2]:
import numpy as np
import pandas as pd

In [3]:
import tweepy
def printmd(string):
    display(Markdown(string))

# Creating API Instance
auth = tweepy.OAuthHandler(api_cred.consumer_key, api_cred.consumer_secret)
auth.set_access_token(api_cred.access_token, api_cred.access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit = True, wait_on_rate_limit_notify = True)

# Authentication of API Instance
try:
    api.verify_credentials()
    printmd('Using Twitter API associated to: {0}'.format('**'+api.me().name+'**'))
    printmd('**Authentication Granted**')
except:
    print('Error: Authenticated Failed')

Using Twitter API associated to: **Dhaval Amodwala**

**Authentication Granted**

In [4]:
# Search Terms
search_words = ['#COVID19', '#CoronaVirus', '#CoronaVaccine', '#CoronaPandemic', '#Lockdown', '#IndiaFightsCorona', '#CoronavirusIndia', '#AtmaNirbharBharat', '#StayHome', '#CoronaWarriors']

In [5]:
import pickle
import jsonpickle

# Downloading Tweets from Twitter
# --------------------------------------------------------------------------

# Iterate through the first 100 Tweets for each HashTags(#)
tweetsPerQuery = 100

# File Path to store Tweets (Json Format)
Json_Data = r'CoronaVirus Tweets\Corona_Twitter_Data_JsonFormat.txt'
# File Path to store Tweets (Actual Tweets)
Actual_Data = r'CoronaVirus Tweets\Corona_Twitter_Data_TextFormat.txt'

since_id = None
max_id = -1
total_tweet = 0
read_tweet = []

print('Downloading the tweeets...Please Wait...\n')

with open(Json_Data, 'w') as outputJson, open(Actual_Data, 'wb') as output:
    for hashtag in search_words:
        tweet_count = 0
        printmd('Downloading hashtag: {0}'.format('**'+hashtag+'**'))
        try:
            if(max_id <= 0):
                if(not since_id):
                    tweets = api.search(q = hashtag, count = tweetsPerQuery, lang = 'en', tweet_mode = 'extended')
                else:
                    tweets = api.search(q = hashtag, count = tweetsPerQuery, lang = 'en', tweet_mode = 'extended', since_id = since_id)
            else:
                if(not since_id):
                    tweets = api.search(q = hashtag, count = tweetsPerQuery, lang = 'en', tweet_mode = 'extended', max_id = str(max_id-1))
                else:
                    tweets = api.search(q = hashtag, count = tweetsPerQuery, lang = 'en', tweet_mode = 'extended', max_id = str(max_id-1), since_id = since_id)
                        
            for tweet in tweets:
                tweet_text = tweet.full_text
                split_words = tweet_text.split()
                if (split_words[0] == 'RT'):
                    try:
                        output_tweet_stmt = tweet.retweeted_status.full_text
                        read_tweet.append(output_tweet_stmt)
                    except:
                        output_tweet_stmt = tweet.full_text
                        read_tweet.append(output_tweet_stmt)
                else:
                    output_tweet_stmt = tweet.full_text
                    read_tweet.append(output_tweet_stmt)
                outputJson.write('HashTag Name: ' + hashtag + '\n' + jsonpickle.encode(tweet._json,unpicklable = False) + '\n\n\n\n')
            tweet_count += len(tweets)
            print('Successfully downloaded {0} tweets'.format(tweet_count))
            total_tweet += tweet_count
        except tweepy.TweepError as e:
            print('Some error!!!: {0}'.format(str(e)))
            break
    pickle.dump(read_tweet, output)

print('\nA total of {0} tweets are downloaded and saved to {1}'.format(total_tweet,Json_Data))

Downloading hashtag: **#COVID19**

Successfully downloaded 100 tweets


Downloading hashtag: **#CoronaVirus**

Successfully downloaded 98 tweets


Downloading hashtag: **#CoronaVaccine**

...

Successfully downloaded 100 tweets


Downloading hashtag: **#StayHome**

Successfully downloaded 100 tweets


Downloading hashtag: **#CoronaWarriors**

Successfully downloaded 100 tweets

A total of 998 tweets are downloaded and saved to CoronaVirus Tweets\Corona_Twitter_Data_JsonFormat.txt


In [6]:
with open(Actual_Data, 'rb') as input_tweet:
    tweet_data = pickle.load(input_tweet)

tweet_data

['Terkini. 8 Jun. Kes baharu hari ini \n\n5️⃣ \n5️⃣ \n6️⃣ \n6️⃣\n\n#COVID19 \n#StayHome',
 'A stitch in time saves nine @narendramodi jee. Good that you realised people should be given vaccine free\n#FreeVaccineForAll #COVID19 #VaccineForAll',
 '@therealdojj @KevinMustafa16 @TfL @SadiqKhan @Heidi_LDN @MichaelMarmot @MalLunam @LeeOdams @MayorofLondon @KeithPrinceAM @marmotihe IMO @MayorofLondon hired @marmotihe @MichaelMarmot to blame PM @BorisJohnson for #Covid19 so that no one’d investigate what @SadiqKhan did/does as @TfL Chair. BUT if more than 50% of 53 London Bus Drivers DIED AFTER 1st WAVE @MayorofLondon should be asked: Were Lessons Learned? https://t.co/XcLwJD12Wz',
 '@therealdojj @KevinMustafa16 @TfL @SadiqKhan @Heidi_LDN @MichaelMarmot @MalLunam @LeeOdams @MayorofLondon @KeithPrinceAM @marmotihe @BorisJohnson &amp; IMO if @SadiqKhan is letting Bus Operators like @ComfortDelGro Metroline &amp; @AbellioLondon SACK Bus Drivers like David O’Sullivan &amp; @BusSafetyRep for report

In [7]:
import re
def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt)        
    return input_txt

def clean_tweets(lst):
    # remove twitter Return handles (RT @xxx:)
    lst = np.vectorize(remove_pattern)(lst, 'RT @[\w]*:')
    # remove twitter handles (@xxx)
    lst = np.vectorize(remove_pattern)(lst, '@[\w]*')
    # remove URL links (httpxxx)
    lst = np.vectorize(remove_pattern)(lst, 'https?://[A-Za-z0-9./]*')
    # remove special characters, numbers, punctuations (except for #)
    lst = np.core.defchararray.replace(lst, '[^a-zA-Z#]', ' ')
    return lst

cleaning_tweet1 = clean_tweets(tweet_data)
cleaning_tweet1

array(['Terkini. 8 Jun. Kes baharu hari ini \n\n5️⃣ \n5️⃣ \n6️⃣ \n6️⃣\n\n#COVID19 \n#StayHome',
       'A stitch in time saves nine  jee. Good that you realised people should be given vaccine free\n#FreeVaccineForAll #COVID19 #VaccineForAll',
       '           IMO  hired   to blame PM  for #Covid19 so that no one’d investigate what  did/does as  Chair. BUT if more than 50% of 53 London Bus Drivers DIED AFTER 1st WAVE  should be asked: Were Lessons Learned? ',
       '            &amp; IMO if  is letting Bus Operators like  Metroline &amp;  SACK Bus Drivers like David O’Sullivan &amp;  for reporting #Covid19 and Safety Risks to Public,  &amp;  Bosses, no one’s “making it up” it’s  POLICY FFS! ',
       '            &amp; IMO if  is letting Bus Operators like  Metroline &amp;  SACK Bus Drivers like David O’Sullivan &amp;  for reporting #Covid19 and Safety Risks to Public,  &amp;  Bosses, no one’s “making it up” it’s  POLICY FFS! ',
       "I fell ill with coronavirus end of March 2020 a

In [8]:
def remove_emoji(inputString):
    emoji_pattern = re.compile('['
                               u'\U0001F1E0-\U0001F1FF'  # flags (iOS)
                               u'\U0001F300-\U0001F5FF'  # symbols & pictographs
                               u'\U0001F600-\U0001F64F'  # emoticons
                               u'\U0001F680-\U0001F6FF'  # transport & map symbols
                               u'\U0001F700-\U0001F77F'  # alchemical symbols
                               u'\U0001F780-\U0001F7FF'  # Geometric Shapes Extended
                               u'\U0001F800-\U0001F8FF'  # Supplemental Arrows-C
                               u'\U0001F900-\U0001F9FF'  # Supplemental Symbols and Pictographs
                               u'\U0001FA00-\U0001FA6F'  # Chess Symbols
                               u'\U0001FA70-\U0001FAFF'  # Symbols and Pictographs Extended-A
                               u'\U00002702-\U000027B0'  # Dingbats
                               u'\U000024C2-\U0001F251'
                            ']+', flags=re.UNICODE)
    return inputString.encode('ascii', 'ignore').decode('ascii')

cleaning_emoji = []
for sen in cleaning_tweet1:
    cleaning_emoji.append(remove_emoji(sen))

cleaning_emoji

['Terkini. 8 Jun. Kes baharu hari ini \n\n5 \n5 \n6 \n6\n\n#COVID19 \n#StayHome',
 'A stitch in time saves nine  jee. Good that you realised people should be given vaccine free\n#FreeVaccineForAll #COVID19 #VaccineForAll',
 '           IMO  hired   to blame PM  for #Covid19 so that no oned investigate what  did/does as  Chair. BUT if more than 50% of 53 London Bus Drivers DIED AFTER 1st WAVE  should be asked: Were Lessons Learned? ',
 '            &amp; IMO if  is letting Bus Operators like  Metroline &amp;  SACK Bus Drivers like David OSullivan &amp;  for reporting #Covid19 and Safety Risks to Public,  &amp;  Bosses, no ones making it up its  POLICY FFS! ',
 '            &amp; IMO if  is letting Bus Operators like  Metroline &amp;  SACK Bus Drivers like David OSullivan &amp;  for reporting #Covid19 and Safety Risks to Public,  &amp;  Bosses, no ones making it up its  POLICY FFS! ',
 "I fell ill with coronavirus end of March 2020 and ended up in hospital. Do you know what it's like to 

In [9]:
from nltk.tokenize import word_tokenize

cleaning_tweet2 = []
for sentence in cleaning_emoji:
    tokens = word_tokenize(sentence)
    words = [word for word in tokens if word.isalnum()]
    temp = ''
    for sen in words:
        temp = '' + temp + ' ' + sen
    a = re.sub('(\\b[A-Za-z] \\b|\\b [A-Za-z]\\b)', '', str(temp))
    cleaning_tweet2.append(' '.join([w for w in a.split() if len(w)>1]))

cleaning_tweet2

['Terkini Jun Kes baharu hari ini COVID19 StayHome',
 'stitch in time saves nine jee Good that you realised people should be given vaccine free FreeVaccineForAll COVID19 VaccineForAll',
 'IMO hired to blame PM for Covid19 so that no oned investigate what as Chair BUT if more than 50 of 53 London Bus Drivers DIED AFTER 1st WAVE should be asked Were Lessons Learned',
 'amp IMO if is letting Bus Operators like Metroline amp SACK Bus Drivers like David OSullivan amp for reporting Covid19 and Safety Risks to Public amp Bosses no ones making it up its POLICY FFS',
 'amp IMO if is letting Bus Operators like Metroline amp SACK Bus Drivers like David OSullivan amp for reporting Covid19 and Safety Risks to Public amp Bosses no ones making it up its POLICY FFS',
 'fell ill with coronavirus end of March 2020 and ended up in hospital Do you know what it like to be on coronavirus ward It was surreal and scary You can read about my experience on any electronic device for 99p coronavirus COVID19',
 'T